In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [149]:
import numpy as np
from pprint import pprint

from collections import Counter, defaultdict

import json
from xgboost import XGBClassifier
import sklearn.tree
from sklearn.metrics import *
from sklearn.model_selection import cross_val_score
from sklearn.externals import joblib

from sample_chapters import *
from feature_extraction import *
from classify import *
from book import *

In [103]:
def extract_texts_and_characters(annotated_data):
    full_characters = np.asarray([datum['character'] for datum in annotated_data])
    full_texts = np.asarray([datum['text'] for datum in annotated_data])
    return full_texts, full_characters



In [125]:
from sklearn.model_selection import KFold

def xval_evaluate(annotated_data, solver, n_splits=10, metric=accuracy_score, mute=True):
    
    full_texts, full_characters = extract_texts_and_characters(annotated_data)
    
    scores = []
    for train_inds, test_inds in KFold(n_splits=n_splits).split(annotated_data):        
        train_ann_data = annotated_data[train_inds]
        test_ann_data = annotated_data[test_inds]

        score = train_test_evaluate(train_ann_data, test_ann_data, solver, n_splits, metric)
        
        if not(mute):
            print(score)
            
        scores.append(score)
    return np.mean(scores, axis=0)


def evaluate(train_ann_data, test_ann_data, solver, n_splits=10, metric=accuracy_score):   
    train_texts, train_characters = extract_texts_and_characters(train_ann_data)
    test_texts, test_characters = extract_texts_and_characters(test_ann_data)

    solver.train(train_texts, train_characters)
    score = solver.test(test_texts, test_characters, metric=metric)
    return score



In [138]:
nicknames2name_comb = {
    "Dany":"Daenerys",
    "Ned" : "Eddard",
    "Sam" : "Samwell",
    "Rollins" : "Pekka"
}

with open("../flat_data/asoif01-04.json","r") as fh:
    ann_ASOIAF = np.asarray(json.load(fh))
    
    
with open("../flat_data/dregs01.json","r") as fh:
    ann_SOC = np.asarray(json.load(fh))
with open("../flat_data/dregs01.json","r") as fh:
    ann_SOC = np.hstack([ann_SOC, np.asarray(json.load(fh))])

ann_comb = np.hstack([ann_SOC, ann_ASOIAF])

np.random.shuffle(ann_ASOIAF)
np.random.shuffle(ann_SOC)
np.random.shuffle(ann_comb)
print("lengths: ", len(ann_comb), len(ann_SOC), len(ann_ASOIAF))
print("POVs: ", *[len(np.unique(extract_texts_and_characters(ann)[1])) for ann in (ann_comb, ann_SOC, ann_ASOIAF)])

lengths:  348 92 256
POVs:  22 7 15


In [137]:
np.unique(extract_texts_and_characters(ann_SOC)[1])

array(['Inej', 'Jesper', 'Joost', 'Kaz', 'Matthias', 'Nina', 'Pekka'],
      dtype='<U8')

In [ ]:
scores = cross_validation_evaluate(ann_ASOIAF, MLCharacterSolver(XGBClassifier(), nicknames2name_comb),
                                  metric=lambda tt,pp: precision_recall_fscore_support(tt,pp, average='macro', warn_for={})[0:3])
np.mean(scores, axis=0)

# Evaluation Program

In [220]:
import pandas as pd

def all_metrics(tt,pp):
    prf = precision_recall_fscore_support(tt,pp, average='weighted', warn_for={})[0:3]
    acc = accuracy_score(tt,pp)
    return np.hstack([prf, acc])

all_metrics_names = ["P", "R", "F1", "Acc"]

## main eval

In [221]:
ML_mdl = MLCharacterSolver(XGBClassifier(), nicknames2name_comb)
FM_mdl = FirstMentionedSolver(nicknames2name_comb)
MC_mdl = MostMentionedSolver(nicknames2name_comb)


program = {
    ("ASOIAF", "ML Classical Features Trained on SOC"): (ann_SOC, ann_ASOIAF, ML_mdl) ,
    ("SOC", "ML Classical Features Trained on ASOIAF"): (ann_ASOIAF, ann_SOC, ML_mdl),   
    ("SOC", "First Mentioned") :                        ([], ann_SOC, FM_mdl),
    ("ASOIAF", "First Mentioned"):                      ([], ann_ASOIAF, FM_mdl),
    ("SOC", "Most Commonly Mentioned"):                 ([], ann_SOC, MC_mdl),
    ("ASOIAF", "Most Commonly Mentioned"):              ([], ann_ASOIAF, MC_mdl),
}


res = pd.DataFrame(index=program.keys(), columns = all_metrics_names)
res.sort_index(inplace=True)

for ind in res.index:
    print(ind, end="")
    score = evaluate(*program[ind], metric=all_metrics)
    res.loc[ind,:] = score
    print(" ", score)
    
res

('ASOIAF', 'First Mentioned')  [0.08571298 0.25       0.12451888 0.25      ]
('ASOIAF', 'ML Classical Features Trained on SOC')  [0.86209911 0.91796875 0.88736337 0.91796875]
('ASOIAF', 'Most Commonly Mentioned')  [0.86042302 0.91015625 0.88016394 0.91015625]
('SOC', 'First Mentioned')  [0.20992534 0.36956522 0.26519616 0.36956522]
('SOC', 'ML Classical Features Trained on ASOIAF')  [0.87439614 0.91304348 0.89258312 0.91304348]
('SOC', 'Most Commonly Mentioned')  [0.73781291 0.7826087  0.74923274 0.7826087 ]


,P,R,F1,Acc
"(ASOIAF, First Mentioned)",0.085713,0.25,0.124519,0.25
"(ASOIAF, ML Classical Features Trained on SOC)",0.862099,0.917969,0.887363,0.917969
"(ASOIAF, Most Commonly Mentioned)",0.860423,0.910156,0.880164,0.910156
"(SOC, First Mentioned)",0.209925,0.369565,0.265196,0.369565
"(SOC, ML Classical Features Trained on ASOIAF)",0.874396,0.913043,0.892583,0.913043
"(SOC, Most Commonly Mentioned)",0.737813,0.782609,0.749233,0.782609


## Cross Evaluation
To test how much it effects things from different styles.

In [222]:
program = {
    ("ASOIAF", "ML Classical Features"):                (ann_ASOIAF, ML_mdl),
    ("SOC", "ML Classical Features"):                   (ann_SOC, ML_mdl),
    ("combined", "ML Classical Features"):              (ann_comb, ML_mdl),
    
    ("SOC", "First Mentioned"):                         (ann_SOC, FM_mdl),
    ("ASOIAF", "First Mentioned"):                      (ann_ASOIAF, FM_mdl),
    ("combined", "First Mentioned"):                    (ann_comb, FM_mdl),
    
    ("SOC", "Most Commonly Mentioned"):                 (ann_SOC, MC_mdl),
    ("ASOIAF", "Most Commonly Mentioned"):              (ann_ASOIAF, MC_mdl),
    ("combined", "Most Commonly Mentioned"):            (ann_comb, MC_mdl),
}

res_xval = pd.DataFrame(index=program.keys(), columns = all_metrics_names)
res_xval.sort_index(inplace=True)

for ind in res_xval.index:
    print(ind, end="")
    score = xval_evaluate(*program[ind], metric=all_metrics) 
    res_xval.loc[ind, "Acc"] = score
    print(" ", score)
    
res_xval

('ASOIAF', 'First Mentioned')  [0.15136777 0.25015385 0.17685377 0.25015385]
('ASOIAF', 'ML Classical Features')  [0.94408791 0.96076923 0.94870154 0.96076923]
('ASOIAF', 'Most Commonly Mentioned')  [0.87444689 0.91       0.88528164 0.91      ]
('SOC', 'First Mentioned')  [0.32287037 0.37       0.32645503 0.37      ]
('SOC', 'ML Classical Features')  [1.         0.97777778 0.98333333 0.97777778]
('SOC', 'Most Commonly Mentioned')  [0.80657407 0.78444444 0.76867725 0.78444444]
('combined', 'First Mentioned')  [0.17205602 0.28168067 0.20472015 0.28168067]
('combined', 'ML Classical Features')  [0.9445042  0.95983193 0.94888329 0.95983193]
('combined', 'Most Commonly Mentioned')  [0.8532521  0.87647059 0.85707483 0.87647059]


,P,R,F1,Acc
"(ASOIAF, First Mentioned)",NaN,NaN,NaN,"[0.15136776556776557, 0.25015384615384617, 0.1..."
"(ASOIAF, ML Classical Features)",NaN,NaN,NaN,"[0.9440879120879121, 0.9607692307692307, 0.948..."
"(ASOIAF, Most Commonly Mentioned)",NaN,NaN,NaN,"[0.8744468864468864, 0.9099999999999999, 0.885..."
"(SOC, First Mentioned)",NaN,NaN,NaN,"[0.32287037037037036, 0.37000000000000005, 0.3..."
"(SOC, ML Classical Features)",NaN,NaN,NaN,"[1.0, 0.9777777777777779, 0.9833333333333334, ..."
"(SOC, Most Commonly Mentioned)",NaN,NaN,NaN,"[0.806574074074074, 0.7844444444444445, 0.7686..."
"(combined, First Mentioned)",NaN,NaN,NaN,"[0.1720560224089636, 0.28168067226890753, 0.20..."
"(combined, ML Classical Features)",NaN,NaN,NaN,"[0.9445042016806722, 0.9598319327731092, 0.948..."
"(combined, Most Commonly Mentioned)",NaN,NaN,NaN,"[0.8532521008403362, 0.876470588235294, 0.8570..."


# Save some trained model

In [ ]:
cls = train_classifier(*extract_texts_and_characters(ann_ASOIAF), 
                       classifier=XGBClassifier())

output_characters = list(run_classifier(extract_texts_and_characters(ann_ASOIAF)[0], 
                       classifier=cls,
                       nicknames2name=nicknames2name_ASOIAF))
reference_characters = [datum['character'] for datum in ann_ASOIAF]

print("acc: ", sklearn.metrics.accuracy_score(output_characters, reference_characters))

joblib.dump(cls, "../trained_models/ASOIAF-no-headings.pkl")


## Feature importance stuff

In [ ]:
_, _,vector_keys = get_feature_vectors(ann_ASOIAF[1]['text'])
feature_weights = list(zip(cls.feature_importances_,vector_keys))
feature_weights.sort(reverse=True)
feature_weights

In [ ]:

scores = evaluate(ann_ASOIAF, nicknames2name_ASOIAF, XGBClassifier(n_estimators=100))
np.mean(scores)

In [ ]:
with open("../flat_data/Warbreaker.json","r") as fh:
    warbreaker = json.load(fh)

In [ ]:
cls = joblib.load("trained_models/ASOIAF-no-headings.pkl")
warbreaker_characters = run_classifier(extract_texts_and_characters(warbreaker)[0], 
                       classifier=cls,)
ann_warbreaker = [(char, datum['text'][1:125]) for char,datum in zip(warbreaker_characters, warbreaker)]
ann_warbreaker

In [ ]:
for imp, name in zip(classifier.feature_importances_, FeatureVec().keys()):
    print(name, "\t", imp )

In [148]:
for a,(b,c) in [tuple()]:
    print(a)

ValueError: need more than 0 values to unpack

In [156]:
texts, _ = load_chapters(load_book("../input_books/unlabelled/warbreaker/Warbreaker.epub"))

gen = MostMentionedSolver().choose_characters(texts)
list(gen)

['[No Characters Detected]',
 'WARBREAKER',
 'Dedication For Emily Sanderson',
 'Kuntry Bumpken',
 'Vasher',
 'Siri',
 'Vivenna',
 'Lightsong',
 'Siri',
 'Vasher',
 'Siri',
 'Lightsong',
 'Siri',
 'Vivenna',
 'Vivenna',
 'Siri',
 'Lightsong',
 'Denth',
 'Vivenna',
 'Siri',
 'Lightsong',
 'Vivenna',
 'Lightsong',
 'Denth',
 'Siri',
 'Vasher',
 'Denth',
 'Mercystar',
 'Siri',
 'Vivenna',
 'Lightsong',
 'Siri',
 'Denth',
 'Denth',
 'Lightsong',
 'Vivenna',
 'Siri',
 'Vivenna',
 'Siri',
 'Denth',
 'Siri',
 'Denth',
 'Lightsong',
 'Denth',
 'Treledees',
 'Vivenna',
 'Allmother',
 'Vivenna',
 'Siri',
 'Of Lifeless',
 'Vasher',
 'Lightsong',
 'Siri',
 'Vivenna',
 'Lightsong',
 'Nightblood',
 'Lightsong',
 'Lightsong',
 'Treledees',
 'Susebron',
 'Vasher',
 'Lightsong',
 'Vivenna',
 'Vivenna',
 'Raoden',
 'Kelsier',
 'Camon',
 'Evil Librarians Alcatraz Author',
 '[No Characters Detected]']

In [ ]:
""